In [1]:
import pandas as pd
import numpy as np
import os
import rioxarray
import geopandas as gpd

In [2]:
def raster_to_gdf_with_date(multiband_raster, Band_name='daily_NDVI'):
    
    image_date = os.path.basename(multiband_raster)[:8]
    
    with rioxarray.open_rasterio(multiband_raster) as raster:
        num_bands = raster.rio.count
        for i in range(1, num_bands+1):
            
            selected_data = raster.isel(band=i-1)
            raster.name = Band_name
            nodata_value = raster.rio.nodata
            #print(nodata_value)
            df = raster.squeeze().to_dataframe().reset_index()
            geometry = gpd.points_from_xy(df.x, df.y)
            gdf = gpd.GeoDataFrame(df, crs=raster.rio.crs, geometry=geometry).to_crs(epsg=4326)
            gdf['latitude'] = gdf.geometry.y
            gdf['longitude'] = gdf.geometry.x

            gdf = gdf.drop(columns=['x', 'y', 'spatial_ref', 'band'], errors='ignore')
            gdf['Date'] = image_date
            
            return gdf[['Date', 'latitude', 'longitude', Band_name, 'geometry']]

In [4]:
# 4 years of data for gatesburg
Gatesburg_2019_PS_dir = os.path.join(os.path.dirname(os.getcwd()), 'Data', 'PlanetScope', 'Gatesburg', '2019')
Gatesburg_2020_PS_dir = os.path.join(os.path.dirname(os.getcwd()), 'Data', 'PlanetScope', 'Gatesburg', '2020')
Gatesburg_2021_PS_dir = os.path.join(os.path.dirname(os.getcwd()), 'Data', 'PlanetScope', 'Gatesburg', '2021')
Gatesburg_2022_PS_dir = os.path.join(os.path.dirname(os.getcwd()), 'Data', 'PlanetScope', 'Gatesburg', '2022')


# test sites
Gatesburg_2024_PS_dir = os.path.join(os.getcwd(), 'Data', 'Gatesburg', '2024')
#US_HWB_2017_PS_dir = os.path.join(os.path.dirname(os.getcwd()), 'Data', 'PlanetScope', 'US-HWB')

In [6]:
# Imputed PS_NDVI directory
PS_ndvi_dir = os.path.join(Gatesburg_2024_PS_dir, 'NDVI', 'standardized_crs_clipped', 'NDVI_imputed')

all_gdfs = []
for ndvi_file in os.listdir(PS_ndvi_dir):
    if ndvi_file.endswith('.tif'):
        ndvi_file_path = os.path.join(PS_ndvi_dir, ndvi_file)
        ndvi_date = ndvi_file[:8]

        all_gdfs.append(raster_to_gdf_with_date(ndvi_file_path, Band_name='daily_NDVI'))

combined_gdf = gpd.GeoDataFrame(pd.concat(all_gdfs, ignore_index=True))
filtered_combined_gdf = combined_gdf[combined_gdf['daily_NDVI']>0]

filtered_combined_gdf.to_csv(os.path.join('NDVI_CSVs', 'daily_PS_NDVI_GBF_2024_entirefield.csv'), index=False)

In [16]:
# Imputed PS_NDVI directory
PS_ndvi_dir = os.path.join(US_HWB_2017_PS_dir, 'NDVI', 'standardized_crs', 'NDVI_imputed')

all_gdfs = []
for ndvi_file in os.listdir(PS_ndvi_dir):
    if ndvi_file.endswith('.tif'):
        ndvi_file_path = os.path.join(PS_ndvi_dir, ndvi_file)
        ndvi_date = ndvi_file[:8]

        all_gdfs.append(raster_to_gdf_with_date(ndvi_file_path, Band_name='daily_NDVI'))

combined_gdf = gpd.GeoDataFrame(pd.concat(all_gdfs, ignore_index=True))
filtered_combined_gdf = combined_gdf[combined_gdf['daily_NDVI']>0]

filtered_combined_gdf.to_csv('daily_PS_NDVI_HWB_2017_entirefield.csv', index=False)